In [17]:
import pandas as pd
import numpy as np
import itertools
import sys
sys.path.insert(0,'../..')
import g4l
import g4l.estimators.ctm
import g4l.tree.generation


In [18]:
# Max tree depth used
max_depth = 4
# Alphabet
A = [0, 1, 2, 3, 4]
# Penalization constant required for BIC
c = 0.5

## 1. Loading the sample

In [19]:
sample = g4l.data.Sample('../example1/folha.txt', A)
initial_tree = g4l.tree.ContextTree(sample, max_depth=max_depth, tree_initialization_method=g4l.tree.generation.incremental_strategy)

Context Trees are appropriately represented by a Data Frame. 
For any given sample `sample`, the following steps are performed:
* Populates Data Frame with all leaf nodes for a maximum depth as indicated in `max_depth`.
* Calculates the number of occurrences of each context (leaf node) in the sample (`node_freq`) 
* Calculates context's probability (`ps`) in the sample
* Calculates probability of transition to each symbol $a \in A$ (`child_probs`).
* Since all these values are computed, we can also compute the likelihood for each context (`lps`).

The resulting initial Data Frame is shown below:

In [20]:
dfx = initial_tree.df
dfx[(dfx.l==4) & (dfx.lps<0)].sort_values(['lps'], ascending=False).head(20)

,node_idx,node,len_idx,node_freq,l,parent_idx,node_prob,lps,transition_sum_log_probs,final,flag
230,230,0334,150,2,4,59,0.004525,-1.386294,-1.386294,0,0
229,229,3342,149,2,4,79,0.666667,-1.386294,-1.386294,0,0
219,219,0034,139,2,4,58,0.006757,-1.386294,-1.386294,0,0
221,221,1343,141,3,4,72,0.250000,-1.909543,-1.504077,0,0
218,218,3433,138,3,4,75,0.272727,-1.909543,-1.504077,0,0
203,203,4300,123,17,4,56,0.040476,-3.803207,-2.893838,0,0
212,212,3034,132,6,4,38,0.004376,-4.158883,-1.386294,0,0
210,210,1342,130,9,4,72,0.750000,-4.767356,-1.755392,0,0
216,216,3430,136,7,4,75,0.636364,-4.780357,-1.406914,0,0
209,209,2134,129,8,4,30,0.012103,-5.292506,-1.450833,0,0


*Implementation notes: In the original algorithm the steps described so far are part of the context tree estimator procedures. These steps were refactored to outside the tree estimation since they always produce the same results for a given sample. In the new implementation, these procedures are hence precomputed and will serve as input to any further call to BIC context tree estimation. (computing time has decreased to about $\frac{1}{10}$ of the original algorithm)*

## 2. Computing the BIC context tree estimator

In [21]:
ctm = g4l.estimators.ctm.CTM(initial_tree)
bic_tree = ctm.execute(c)

# Show resulting tree
print('Resulting tree: ', bic_tree.to_str())

Resulting tree:  00 0001 0010 0020 003 0030 0103 0120 0201 0303 0420 1020 1030 1201 13 130 1303 1420 2 2001 2010 21 210 2103 2120 3020 3030 320 3201 33 330 3303 3420 4 4201 43 430 4303


The CTM procedure computes BIC context tree estimator efficiently. It receives as parameter the penalization constant `c` and returns a DataFrame representing the tree, containing all leaf nodes for a maximum depth as indicated in `max_depth`.  For any given `sample`, the DataFrame initially calculates the number of occurrences of each leaf node in the sample (`node_freq`) along with its probability `ps` and the probability of transition to each symbol $a \in A$ (`child_probs`). Since these values are computed, we can also compute the likelihood for each context `lps`.

In [22]:
bic_tree.df

,node_idx,node,len_idx,node_freq,l,parent_idx,node_prob,lps,transition_sum_log_probs,final,flag
17,17,00,11,10519,2,2,0.238467,-11239.564488,-11.977777,1,0
129,129,0001,49,2360,4,54,0.714502,-2402.353929,-7.482955,1,0
123,123,0010,43,3598,4,49,0.615990,-3355.740144,-7.805862,1,0
109,109,0020,29,571,4,45,0.616631,-418.451227,-1.387402,1,0
58,58,003,33,296,3,17,0.028140,-293.007993,-9.053958,1,0
181,181,0030,101,187,4,58,0.631757,-192.917428,-8.620047,1,0
92,92,0103,12,1714,4,35,0.195729,-1442.588257,-9.599694,1,0
101,101,0120,21,1858,4,39,0.628552,-1306.969986,-1.390474,1,0
88,88,0201,8,4696,4,33,0.543896,-4010.074481,-8.180476,1,0
98,98,0303,18,794,4,37,0.238367,-682.018090,-9.336676,1,0


### 2.1 - Executing the BIC tree estimator

The following steps demonstrate in details how BIC tree estimator performs to achieve the outcome data frame described above.

In [23]:
# Let's consider the initial tree with freqs/probs pre-calculated for the input Sample
data_frame = initial_tree.df.copy()

# These are the first 8 nodes in the Data Frame:
data_frame.head(8)

,node_idx,node,len_idx,node_freq,l,parent_idx,node_prob,lps,transition_sum_log_probs,final,flag
0,0,2,0,21830,1,None,0.223323,-14715.435172,-1.424897,0,0
1,1,1,1,21830,1,None,0.223323,-18794.382713,-8.296834,0,0
2,2,0,2,44111,1,None,0.451259,-62840.152905,-9.367255,0,0
3,3,3,3,7909,1,None,0.080910,-6671.568838,-9.670055,0,0
4,4,4,4,2070,1,None,0.021176,-1333.215732,-9.133726,0,0
5,5,,5,1,1,None,0.000010,0.000000,0.000000,0,0
6,6,21,0,8791,2,0,0.402703,-7026.888299,-8.864123,0,0
7,7,10,1,15136,2,1,0.693358,-14916.737004,-7.272850,0,0


In [25]:
# We instantiate the Context Tree Maximizer class by passing the pre-calculated tree as parameter
ctm = g4l.estimators.ctm.CTM(initial_tree)

### 2.2 - Applying the penalization

We apply a penalization considering the constant $c$ (in this case `c = 0.5`). We update each context's lps value as following: 

`data_frame.lps -= np.log(n) * (degrees_of_freedom * c)`

with  `degrees_of_freedom` = $|A|-1$

In [26]:
ctm.apply_penalization(c, data_frame)
data_frame

,node_idx,node,len_idx,node_freq,l,parent_idx,node_prob,lps,transition_sum_log_probs,final,flag
0,0,2,0,21830,1,None,0.223323,-14738.415530,-1.424897,0,0
1,1,1,1,21830,1,None,0.223323,-18817.363070,-8.296834,0,0
2,2,0,2,44111,1,None,0.451259,-62863.133263,-9.367255,0,0
3,3,3,3,7909,1,None,0.080910,-6694.549196,-9.670055,0,0
4,4,4,4,2070,1,None,0.021176,-1356.196089,-9.133726,0,0
...,...,...,...,...,...,...,...,...,...,...,...
228,228,3334,148,1,4,68,0.010309,-22.980357,0.000000,0,0
229,229,3342,149,2,4,79,0.666667,-24.366652,-1.386294,0,0
230,230,0334,150,2,4,59,0.004525,-24.366652,-1.386294,0,0
231,231,4214,151,1,4,61,0.001656,-22.980357,0.000000,0,0


### 2.3 Selecting candidate contexts

For every context that occurs more than once in the sample, we sum the  `lps` of its child contexts and compare to its own `lps`. When the first value is greater than the second, we update the context's `lps` with the new value and "flag" this context to be further removed.

In [27]:
ctm.block2(data_frame) # TODO: rename this method
data_frame

,node_idx,node,len_idx,node_freq,l,parent_idx,node_prob,lps,transition_sum_log_probs,final,flag
0,0,2,0,21830,1,None,0.223323,-14738.415530,-1.424897,0,0
1,1,1,1,21830,1,None,0.223323,-7263.807918,-8.296834,0,1
2,2,0,2,44111,1,None,0.451259,-14154.087852,-9.367255,0,1
3,3,3,3,7909,1,None,0.080910,-783.366441,-9.670055,0,1
4,4,4,4,2070,1,None,0.021176,-1356.196089,-9.133726,0,0
...,...,...,...,...,...,...,...,...,...,...,...
228,228,3334,148,1,4,68,0.010309,-22.980357,0.000000,0,0
229,229,3342,149,2,4,79,0.666667,-24.366652,-1.386294,0,0
230,230,0334,150,2,4,59,0.004525,-24.366652,-1.386294,0,0
231,231,4214,151,1,4,61,0.001656,-22.980357,0.000000,0,0


### 2.4 Selecting the final nodes

*Need help to describe this step*

In [28]:
ctm.block3(data_frame)

In [29]:
data_frame

,node_idx,node,len_idx,node_freq,l,parent_idx,node_prob,lps,transition_sum_log_probs,final,flag
0,0,2,0,21830,1,None,0.223323,-14738.415530,-1.424897,1,0
1,1,1,1,21830,1,None,0.223323,-7263.807918,-8.296834,0,1
2,2,0,2,44111,1,None,0.451259,-14154.087852,-9.367255,0,1
3,3,3,3,7909,1,None,0.080910,-783.366441,-9.670055,0,1
4,4,4,4,2070,1,None,0.021176,-1356.196089,-9.133726,1,0
...,...,...,...,...,...,...,...,...,...,...,...
228,228,3334,148,1,4,68,0.010309,-22.980357,0.000000,0,0
229,229,3342,149,2,4,79,0.666667,-24.366652,-1.386294,0,0
230,230,0334,150,2,4,59,0.004525,-24.366652,-1.386294,0,0
231,231,4214,151,1,4,61,0.001656,-22.980357,0.000000,0,0


### 2.4 - Resulting context tree

In [30]:
ctm.final_tree(data_frame)

,node_idx,node,len_idx,node_freq,l,parent_idx,node_prob,lps,transition_sum_log_probs,final,flag
17,17,00,11,10519,2,2,0.238467,-11239.564488,-11.977777,1,0
129,129,0001,49,2360,4,54,0.714502,-2402.353929,-7.482955,1,0
123,123,0010,43,3598,4,49,0.615990,-3355.740144,-7.805862,1,0
109,109,0020,29,571,4,45,0.616631,-418.451227,-1.387402,1,0
58,58,003,33,296,3,17,0.028140,-293.007993,-9.053958,1,0
181,181,0030,101,187,4,58,0.631757,-192.917428,-8.620047,1,0
92,92,0103,12,1714,4,35,0.195729,-1442.588257,-9.599694,1,0
101,101,0120,21,1858,4,39,0.628552,-1306.969986,-1.390474,1,0
88,88,0201,8,4696,4,33,0.543896,-4010.074481,-8.180476,1,0
98,98,0303,18,794,4,37,0.238367,-682.018090,-9.336676,1,0


### String representation

The resulting object also provides a string representation of the selected context tree:

In [31]:
bic_tree.to_str()

'00 0001 0010 0020 003 0030 0103 0120 0201 0303 0420 1020 1030 1201 13 130 1303 1420 2 2001 2010 21 210 2103 2120 3020 3030 320 3201 33 330 3303 3420 4 4201 43 430 4303'

### Comparing context trees

A handy comparison method is available to compare trees

In [32]:
bic_tree.equals_to(bic_tree)

True

In [34]:
tree1 = g4l.estimators.ctm.CTM(initial_tree).execute(0.2)    # c = 0.5
tree2 = g4l.estimators.ctm.CTM(initial_tree).execute(3000)   # c = 3000
tree3 = g4l.estimators.ctm.CTM(initial_tree).execute(4000)   # c = 4000

print("tree1:", tree1.to_str(), "\n")
print("tree2:", tree2.to_str(), "\n")
print("tree3:", tree3.to_str(), "\n\n")

print("tree1 == tree2 ?", tree1.equals_to(tree2))
print("tree1 == tree3 ?", tree1.equals_to(tree3))
print("tree2 == tree3 ?", tree2.equals_to(tree3))

tree1: 00 0001 0010 0020 003 0030 0103 0120 0201 0303 0420 1020 1030 1201 13 130 1303 1420 2 2001 2010 21 210 2103 2120 3020 3030 320 3201 33 330 3303 3420 4 4201 43 430 4303 

tree2: 0 1 2 3 4 

tree3: 0 1 2 3 4 


tree1 == tree2 ? False
tree1 == tree3 ? False
tree2 == tree3 ? True


In [35]:
bic_tree.log_likelihood()

-82147.8701857751